In [1]:
import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
import sys
sys.path.append("..")
%cd ..


/Users/AsgerSturisTang/OneDrive - Danmarks Tekniske Universitet/DTU/6. Semester/Bachelor2021


In [2]:
from DataPrep.ImportData import importer
from Modelling import modelling
import keras
from keras import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Input, Reshape
import pyforest
import torch
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error



In [3]:
m = modelling()
df = m.df.drop(columns=["Start Date","Charging Time (mins)", "Total Duration (mins)", "Port Number"])

In [4]:
cols_to_standardize = ['# Professional & Other Places', '# Food', '# Shop & Service',
       '# Travel & Transport', '# Outdoors & Recreation',
       '# Arts & Entertainment', '# Nightlife Spot', '# Residence',
       '# College & University', '# Event']

In [5]:
df.head()

,Label,Energy (kWh),CenterLon,CenterLat,Level 1,Level 2,Energy (kWh)_lag1,Energy (kWh)_lag2,Energy (kWh)_lag3,Energy (kWh)_lag4,...,Year_Month_5,Year_Month_6,Year_Month_7,Year_Month_8,Year_Month_9,Year_Month_10,Year_Month_11,Year_Month_12,is_holiday,is_weekend
0,0.0,1.679882,-122.16,37.445,0,8,2.098609,1.260311,2.325573,1.726841,...,0,0,0,1,0,0,0,0,0,0
1,0.0,1.899242,-122.16,37.445,0,5,1.679882,2.098609,1.260311,2.325573,...,0,0,0,1,0,0,0,0,0,0
2,0.0,1.574471,-122.16,37.445,0,10,1.899242,1.679882,2.098609,1.260311,...,0,0,0,1,0,0,0,0,0,0
3,0.0,1.982032,-122.16,37.445,0,14,1.574471,1.899242,1.679882,2.098609,...,0,0,0,1,0,0,0,0,0,1
4,0.0,2.232860,-122.16,37.445,0,3,1.982032,1.574471,1.899242,1.679882,...,0,0,0,1,0,0,0,0,0,1


In [6]:
sc = StandardScaler()
stand_poi = sc.fit_transform(df[cols_to_standardize])

In [7]:
stand_poi = pd.DataFrame(stand_poi, index=df.index, columns=cols_to_standardize)

In [8]:
for i in cols_to_standardize:
    df[i] = stand_poi[i]

In [9]:
df["Label"] = df["Label"] + 1

In [10]:
df = df.fillna(0)

In [11]:
org = df["Energy (kWh)"].copy()


In [12]:
df_org = df.copy()

In [13]:

df["Energy (kWh)"] = df["Energy (kWh)"].diff().diff()
df = df.dropna()


In [63]:
def toNormal(target, original, training = None):
    if training != None:
        target = pd.DataFrame(target, index = training.index)
        training = training.sort_index()

    target = target.sort_index()
    min_index = target.index.min()
    diff = original.iloc[min_index - 1]
    first_org_value = original.iloc[min_index - 2]
    diff = diff - first_org_value
    target[1] = diff
    target = target.sort_index()
    target = target.cumsum()
    target[0] = first_org_value
    target = target.sort_index()

    if training != None:
        training[1] = diff
        training = training.sort_index()
        training = training.cumsum()
        training[0] = first_org_value
        training = training.sort_index()

        return target.cumsum(), training.cumsum()


    return target.cumsum()
        


In [62]:
toNormal(df["Energy (kWh)"], org)

0        1.679882
1        1.899242
2        1.574471
3        1.982032
4        2.232860
           ...   
17276    4.018583
17277    4.247111
17278    2.646125
17279    4.348600
17280    3.383808
Name: Energy (kWh), Length: 17281, dtype: float64